In [22]:
import pandas as pd
import numpy as np


In [23]:
dfdis =  pd.read_csv(r"districts.csv")

In [24]:
dfdis.head()

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested
0,2020-04-26,Andaman and Nicobar Islands,Unknown,33,11,0,0,NaN
1,2020-04-26,Andhra Pradesh,Anantapur,53,14,4,0,NaN
2,2020-04-26,Andhra Pradesh,Chittoor,73,13,0,0,NaN
3,2020-04-26,Andhra Pradesh,East Godavari,39,12,0,0,NaN
4,2020-04-26,Andhra Pradesh,Guntur,214,29,8,0,NaN


In [7]:
dfwise = pd.read_csv(r"district_wise.csv")

In [26]:

dfwise = dfwise.loc[:,['District_Key','District']]
dfwise.head()

,District_Key,District
0,UN_Unassigned,Unassigned
1,AN_Nicobars,Nicobars
2,AN_North and Middle Andaman,North and Middle Andaman
3,AN_South Andaman,South Andaman
4,AP_Foreign Evacuees,Foreign Evacuees


In [27]:
# list of district code is created
dis_code = list(dfwise.District_Key.unique())

In [28]:
# Daily active cases was determined by  subtracting confirmed cases from recovered and daily deaths
dfdis.loc[:,'cases'] = 0

In [29]:
dfdis.loc[:,'cases'] = dfdis.loc[:,'Confirmed'] - dfdis.loc[:,'Recovered'] - dfdis.loc[:,'Deceased']

In [30]:
dfdis.head()

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested,cases
0,2020-04-26,Andaman and Nicobar Islands,Unknown,33,11,0,0,NaN,22
1,2020-04-26,Andhra Pradesh,Anantapur,53,14,4,0,NaN,35
2,2020-04-26,Andhra Pradesh,Chittoor,73,13,0,0,NaN,60
3,2020-04-26,Andhra Pradesh,East Godavari,39,12,0,0,NaN,27
4,2020-04-26,Andhra Pradesh,Guntur,214,29,8,0,NaN,177


In [31]:
dfdis = dfdis.drop(['State','Recovered','Deceased','Other','Tested'],axis=1)

In [32]:
dfdis  = dfdis.loc[:,['District','Date','cases']]

In [33]:
# As covid data was only available form 26-4-2020, so we created dummy data from 15-3-2020 to 25-3-2020
import datetime
lis = list(dfdis['District'].unique())
start_date = datetime.date(2020, 3, 15)
end_date = datetime.date(2020, 4, 25)
# we decremented date by 1
delta = datetime.timedelta(days=1)
while start_date <= end_date:
    #created a temp data frame and stored the dummy data for a particular day
    temp=pd.DataFrame(dict({'Date': end_date,'District': lis,'cases':0}))
    dfdis =pd.concat([temp,dfdis], axis=0)
    #print(end_date)
    end_date -= delta
    

In [34]:
# For pandas resampling process, date column was made index of dtat frame
dfdis=dfdis.set_index("Date")
dfdis.index = pd.to_datetime(dfdis.index)

In [35]:
# data after 14-08-2021 is discared
dfdis = dfdis.loc['2020-03-15':'2021-08-15',:]

In [36]:
dfdis2 = dfdis
dfdis3 = dfdis

In [37]:
# dis stores district name for a district code
dis = dfwise[dfwise['District_Key']==dis_code[0]]['District']
dis = list(dis)[0]
# for a particular  district  a separate data frame is created which stores weekly covid active cases
temp = dfdis[dfdis.loc[:,'District']==dis]
temp = temp.resample('W',closed="right", label="right").max()
temp.loc[:,'District'] = dis_code[0]
temp = temp.reset_index()
l = temp.shape[0]
# values of date column were replaced  by integeres 1,2,3.., which represent number of week

temp.loc[:,'Date'] = range(1,l+1)


In [42]:
temp.head()

,Date,District,cases
0,1.0,AP_Foreign Evacuees,0.0
1,2.0,AP_Foreign Evacuees,0.0
2,3.0,AP_Foreign Evacuees,0.0
3,4.0,AP_Foreign Evacuees,0.0
4,5.0,AP_Foreign Evacuees,0.0


In [39]:
# as active cases for first district is already detemined , we removed that district name from  the dis_code file
dis_code = dis_code[1:]


In [40]:
# above process was repeated for each district
for i in dis_code:
    dis = dfwise[dfwise['District_Key']==i]['District']
    dis = list(dis)[0]
    temp2 = dfdis[dfdis.loc[:,'District']==dis]
    temp2 = temp2.resample('W',closed="right", label="right").max()
    temp2.loc[:,'District'] = i
    temp2 = temp2.reset_index()
    l = temp2.shape[0]
    temp2.loc[:,'Date'] = range(1,l+1)
    temp =pd.concat([temp,temp2], axis=0)


In [41]:
temp.shape

(52202, 3)

In [43]:
temp.rename(columns = {'Date':'timeid','District':'districtid'}, inplace = True)

In [44]:
temp = temp.loc[:,['districtid','timeid','cases']]

In [45]:
temp.to_csv('cases-week.csv', index=False)

In [46]:
dfdis2.head()

,District,cases
Date,,
2020-03-15,Unknown,0
2020-03-15,Anantapur,0
2020-03-15,Chittoor,0
2020-03-15,East Godavari,0
2020-03-15,Guntur,0


In [18]:
# Daily active cases was determined by  subtracting confirmed cases from recovered and daily deaths
dfdis2.loc[:,'cases'] = dfdis2.loc[:,'Confirmed'] - dfdis2.loc[:,'Recovered'] - dfdis.loc[:,'Deceased']


In [19]:
dfdis2 = dfdis2.iloc[:,[0,2,8]]

In [20]:
dfwise2 = dfwise2.loc[:,['District_Key','District']]

In [47]:
dis_code = list(dfwise.District_Key.unique())

In [48]:
# dis stores district name for a district code
dis = dfwise[dfwise['District_Key']==dis_code[0]]['District']
dis = list(dis)[0]
# for a particular  district  a separate data frame is created which stores weekly covid active cases
temp = dfdis[dfdis.loc[:,'District']==dis]
# resmapling was done by month
temp = temp.resample('M',closed="right", label="right").max()
temp.loc[:,'District'] = dis_code[0]
temp = temp.reset_index()
l = temp.shape[0]
# values of date column were replaced  by integeres 1,2,3.., which represent number of week

temp.loc[:,'Date'] = range(1,l+1)

In [49]:
dis_code = dis_code[1:]

In [50]:
for i in dis_code:
    dis = dfwise[dfwise['District_Key']==i]['District']
    dis = list(dis)[0]
    temp2 = dfdis[dfdis.loc[:,'District']==dis]
    temp2 = temp2.resample('M',closed="right", label="right").max()
    temp2.loc[:,'District'] = i
    temp2 = temp2.reset_index()
    l = temp2.shape[0]
    temp2.loc[:,'Date'] = range(1,l+1)
    temp =pd.concat([temp,temp2], axis=0)


In [53]:
temp.head()

,Date,District,cases
0,1.0,AP_Foreign Evacuees,0.0
1,2.0,AP_Foreign Evacuees,0.0
2,3.0,AP_Foreign Evacuees,111.0
3,4.0,AP_Foreign Evacuees,297.0
4,5.0,AP_Foreign Evacuees,302.0


In [54]:
temp.rename(columns = {'Date':'timeid','District':'districtid'}, inplace = True)

In [55]:
temp = temp.loc[:,['districtid','timeid','cases']]

In [56]:
temp.to_csv('cases-month.csv', index=False)

In [67]:
dfdis3.head()

,timeid,districtid,cases
0,2021-08-14,Unknown,2
1,2021-08-14,Anantapur,234
2,2021-08-14,Chittoor,2550
3,2021-08-14,East Godavari,3122
4,2021-08-14,Foreign Evacuees,0


In [59]:
dfdis3 = dfdis3.loc['2021-08-14',:]

In [64]:
dfdis3 = dfdis3.reset_index()


In [66]:
dfdis3.rename(columns = {'Date':'timeid','District':'districtid'}, inplace = True)

In [68]:
dfdis3.to_csv('cases-overall.csv', index=False)
    